### 3 TensorFlow的层次结构-3.3 高阶API示范
下面的范例使用TensorFlow的高阶API实现线性回归模型。

TensorFlow的高阶API主要为tf.keras.models提供的模型的类接口。

使用Keras接口有以下3种方式构建模型：使用Sequential按层顺序构建模型，使用函数式API构建任意结构模型，继承Model基类构建自定义模型。

此处分别演示使用Sequential按层顺序构建模型以及继承Model基类构建自定义模型。

In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers, metrics, losses, optimizers

#样本数量
n = 800

# 生成测试用数据集
X = tf.random.uniform([n,2],minval=-10,maxval=10) 
w0 = tf.constant([[2.0],[-1.0]])
b0 = tf.constant(3.0)

Y = X@w0 + b0 + tf.random.normal([n, 1], mean=0.0, stddev=2.0)

In [2]:
tf.keras.backend.clear_session()


linear = models.Sequential()
linear.add(layers.Dense(1, input_shape=(2, )))
linear.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [3]:
linear.compile(optimizer='adam', loss='mse', metrics=['mae'])
linear.fit(X,Y, batch_size=30, epochs=100, validation_split=0.2, verbose=1, )
tf.print('w= ', linear.layers[0].kernel)
tf.print('b= ', linear.layers[0].bias)

Train on 640 samples, validate on 160 samples
Epoch 1/100
640/640 [==============================] - 1s 2ms/sample - loss: 195.3712 - mae: 11.5559 - val_loss: 224.0122 - val_mae: 12.6300
Epoch 2/100
640/640 [==============================] - 0s 351us/sample - loss: 190.8137 - mae: 11.4191 - val_loss: 218.5587 - val_mae: 12.4737
Epoch 3/100
640/640 [==============================] - 0s 166us/sample - loss: 186.2322 - mae: 11.2824 - val_loss: 213.2960 - val_mae: 12.3209
Epoch 4/100
640/640 [==============================] - 0s 186us/sample - loss: 181.8121 - mae: 11.1470 - val_loss: 208.0451 - val_mae: 12.1665
Epoch 5/100
640/640 [==============================] - 0s 168us/sample - loss: 177.4322 - mae: 11.0115 - val_loss: 202.8871 - val_mae: 12.0129
Epoch 6/100
640/640 [==============================] - 0s 227us/sample - loss: 173.1414 - mae: 10.8775 - val_loss: 197.8825 - val_mae: 11.8619
Epoch 7/100
640/640 [==============================] - 0s 412us/sample - loss: 168.9163 - mae: 10.

Epoch 59/100
640/640 [==============================] - 0s 66us/sample - loss: 39.2545 - mae: 5.1524 - val_loss: 42.3812 - val_mae: 5.3896
Epoch 60/100
640/640 [==============================] - 0s 64us/sample - loss: 38.0600 - mae: 5.0684 - val_loss: 40.9434 - val_mae: 5.2926
Epoch 61/100
640/640 [==============================] - 0s 63us/sample - loss: 36.8470 - mae: 4.9857 - val_loss: 39.6354 - val_mae: 5.2041
Epoch 62/100
640/640 [==============================] - 0s 61us/sample - loss: 35.7231 - mae: 4.9052 - val_loss: 38.3125 - val_mae: 5.1136
Epoch 63/100
640/640 [==============================] - 0s 61us/sample - loss: 34.6039 - mae: 4.8249 - val_loss: 37.0655 - val_mae: 5.0267
Epoch 64/100
640/640 [==============================] - 0s 60us/sample - loss: 33.5341 - mae: 4.7467 - val_loss: 35.8229 - val_mae: 4.9384
Epoch 65/100
640/640 [==============================] - 0s 59us/sample - loss: 32.4827 - mae: 4.6687 - val_loss: 34.6466 - val_mae: 4.8534
Epoch 66/100
640/640 [=====

In [4]:
tf.print("w = ",linear.layers[0].kernel)
tf.print("b = ",linear.layers[0].bias)

w =  [[1.65349638]
 [-0.814564228]]
b =  [1.57031238]


#### 继承Model基类构建自定义模型【面向专家】

In [5]:
#打印时间分割线
@tf.function
def printbar():
    ts = tf.timestamp()
    today_ts = ts%(24*60*60)

    hour = tf.cast(today_ts//3600+8,tf.int32)%tf.constant(24)
    minite = tf.cast((today_ts%3600)//60,tf.int32)
    second = tf.cast(tf.floor(today_ts%60),tf.int32)

    def timeformat(m):
        if tf.strings.length(tf.strings.format("{}",m))==1:
            return(tf.strings.format("0{}",m))
        else:
            return(tf.strings.format("{}",m))

    timestring = tf.strings.join([timeformat(hour),timeformat(minite),
                timeformat(second)],separator = ":")
    tf.print("=========="*8,end = "")
    tf.print(timestring)

In [6]:
ds_train = tf.data.Dataset.from_tensor_slices((X[0:n*3//4,:],Y[0:n*3//4,:])) \
     .shuffle(buffer_size = 1000).batch(20) \
     .prefetch(tf.data.experimental.AUTOTUNE) \
     .cache()

ds_valid = tf.data.Dataset.from_tensor_slices((X[n*3//4:,:],Y[n*3//4:,:])) \
     .shuffle(buffer_size = 1000).batch(20) \
     .prefetch(tf.data.experimental.AUTOTUNE) \
     .cache()

In [7]:
tf.keras.backend.clear_session()

class MyModel(models.Model):
    def __init__(self):
        super(MyModel, self).__init__()

    def build(self,input_shape):
        self.dense1 = layers.Dense(1)   
        super(MyModel,self).build(input_shape)

    def call(self, x):
        y = self.dense1(x)
        return(y)
    
model = MyModel()
model.build(input_shape =(None,2))
model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [10]:
### 自定义训练循环(专家教程)
optimizer = optimizers.Adam()
loss_func = losses.MeanSquaredError()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_metric = tf.keras.metrics.MeanAbsoluteError(name='train_mae')

valid_loss = tf.keras.metrics.Mean(name='valid_loss')
valid_metric = tf.keras.metrics.MeanAbsoluteError(name='valid_mae')


@tf.function
def train_step(model, features, labels):
    with tf.GradientTape() as tape:
        preditions = model(features)
        loss = loss_func(labels, preditions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss.update_state(loss)
    train_metric.update_state(labels, preditions)
    

@tf.function
def valid_step(model, features, labels):
    predictions = model(features)
    batch_loss = loss_func(labels, predictions)
    valid_loss.update_state(batch_loss)
    valid_metric.update_state(labels, predictions)

@tf.function
def train_model(model,ds_train,ds_valid,epochs):
    for epoch in tf.range(1,epochs+1):
        for features, labels in ds_train:
            train_step(model,features,labels)

        for features, labels in ds_valid:
            valid_step(model,features,labels)

        logs = 'Epoch={},Loss:{},MAE:{},Valid Loss:{},Valid MAE:{}'

        if  epoch%100 ==0:
            printbar()
            tf.print(tf.strings.format(
                logs,
                (epoch,train_loss.result(),train_metric.result(),
                 valid_loss.result(),valid_metric.result())))
            tf.print("w=",model.layers[0].kernel)
            tf.print("b=",model.layers[0].bias)
            tf.print("")

        train_loss.reset_states()
        valid_loss.reset_states()
        train_metric.reset_states()
        valid_metric.reset_states()

In [12]:
train_model(model, ds_train, ds_valid, 2000)

================================================================================20:31:12
Epoch=100,Loss:4.04551268,MAE:1.62805414,Valid Loss:3.70969,Valid MAE:1.56058216
w= [[1.97429454]
 [-0.99056983]]
b= [3.12274528]

================================================================================20:31:26
Epoch=200,Loss:4.04555178,MAE:1.62804198,Valid Loss:3.70972252,Valid MAE:1.56059325
w= [[1.97429442]
 [-0.99056983]]
b= [3.12275]

================================================================================20:31:41
Epoch=300,Loss:4.04556179,MAE:1.6280458,Valid Loss:3.70971131,Valid MAE:1.56059158
w= [[1.97429442]
 [-0.99056983]]
b= [3.12275]

================================================================================20:31:52
Epoch=400,Loss:4.04553175,MAE:1.62806559,Valid Loss:3.70970058,Valid MAE:1.56059015
w= [[1.97429442]
 [-0.99056983]]
b= [3.12275]

================================================================================20:32:10
Epoch=500,Loss:4.04551411,MAE:1.